In [1]:
from llm_manager import get_llm_instance
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.base import RunnableLambda, RunnableParallel
from langchain_core.runnables.passthrough import RunnablePassthrough
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
import re, json

F:\GenAI\LangGraph_OpenSource\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. LLM model

In [2]:
llm_engine = get_llm_instance()

Initializing LLM: mistralai/Mistral-7B-Instruct-v0.2...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.40s/it]
Device set to use cuda


LLM initialization complete for mistralai/Mistral-7B-Instruct-v0.2.


F:\GenAI\LangGraph_OpenSource\llm_manager.py:29: UserWarning: LLMManager is already initialized. Returning existing instance.
  warnings.warn("LLMManager is already initialized. Returning existing instance.")


# 2. Output Parser 

In [3]:
string_parser = StrOutputParser()

# 3. State Define

In [4]:
class AnalysisState(TypedDict):
    text: str
    sentiment: str
    summary: str

# 4. Node Define

In [5]:
def extract_json_sentiment(llm_output: str) -> str:
    # Expect a string, not a dict
    matches = re.findall(r"\{[^{}]*\}", llm_output)
    if not matches:
        return ""
    try:
        parsed_json = json.loads(matches[-1])
        print("Sentiment parsed:", parsed_json.get("sentiment", ""))
        return parsed_json.get("sentiment", "")
    except Exception as e:
        print("JSON parse failed:", e)
        return ""

In [6]:
def sentiment_node(state: AnalysisState):
    user_text = state["text"]
    
    prompt = PromptTemplate(
        template=(
            """
            Classify the sentiment as positive or negative of the following text \n {text}
    
            The required format should be:
            {{
                "sentiment":"positive or negative"
            }}
            """
        ),
        input_variables=["text"]
    )
    
    # Pass LLM output string through the extraction lambda
    chain = prompt | llm_engine | string_parser | RunnableLambda(extract_json_sentiment)
    
    sentiment = chain.invoke({"text": user_text})
    
    return {"sentiment": sentiment}

In [7]:
def summary_node(state: AnalysisState):

    sentiment_label = state["sentiment"]
    
    prompt = PromptTemplate(
        template="Compose a {sentiment} remark about AI.",
        input_variables=["sentiment"]
    )
    
    chain = prompt | llm_engine | string_parser
    
    summary = chain.invoke({"sentiment": sentiment_label})
    
    return {"summary": summary}

# 5. Graph Design

In [8]:
analysis_graph = StateGraph(AnalysisState)

analysis_graph.add_node("sentiment_node", sentiment_node)
analysis_graph.add_node("summary_node", summary_node)

analysis_graph.add_edge(START, "sentiment_node")
analysis_graph.add_edge("sentiment_node", "summary_node")
analysis_graph.add_edge("summary_node", END)

engine = analysis_graph.compile()

# 6. Execution

In [9]:
initial_input = {"text": "AI isn’t always reliable and can sometimes give incorrect answers."}
output_state = engine.invoke(initial_input)
print(output_state)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentiment parsed: neutral
{'text': 'AI isn’t always reliable and can sometimes give incorrect answers.', 'sentiment': 'neutral', 'summary': '<s> [INST] Compose a neutral remark about AI. [/INST] Artificial Intelligence (AI) is a branch of computer science that focuses on creating intelligent machines capable of performing tasks that would normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. It is a rapidly advancing field with numerous potential applications in industries ranging from healthcare to finance to transportation, and its development continues to generate significant interest and debate among experts and the public.'}
